In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri. June 25 2021

@author: Jane Hsieh

@Folder stursure of data:
    data_folder (dir:origion_path)
    |__label1 folder
    |              |__ image11 file
    |              |__ image12 file
    |              | ...
    |               
    |__label2 folder 
    |              |__ image21 file
    |              |__ image22 file
    |              | ...
    |
    |__label3 folder
    ...
@Issue:
    1. ResourceExhaustedError解决方法: https://blog.csdn.net/lics999/article/details/78517638
       --> type "invidia-smi" on terminal
"""

'\nCreated on Fri. June 25 2021\n\n@author: Jane Hsieh\n\n@Folder stursure of data:\n    data_folder (dir:origion_path)\n    |__label1 folder\n    |              |__ image11 file\n    |              |__ image12 file\n    |              | ...\n    |               \n    |__label2 folder \n    |              |__ image21 file\n    |              |__ image22 file\n    |              | ...\n    |\n    |__label3 folder\n    ...\n@Issue:\n    1. ResourceExhaustedError解决方法: https://blog.csdn.net/lics999/article/details/78517638\n       --> type "invidia-smi" on terminal\n'

In [2]:
!/usr/bin/python -m pip install --upgrade pip

In [3]:
!pip install -U scikit-learn 
# !pip install scikit-learn==1.0.2

In [4]:
!pip install -U imblearn 
# !pip install imblearn --ignore-installed scikit-learn

In [5]:
!pip install Keras==2.2.4

In [6]:
!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1

# !pip install tensorflow
# !pip install tensorflow-gpu

UnboundLocalError: local variable 'child' referenced before assignment

In [ ]:
!pip install opencv-python-headless # !pip install opencv-python

In [ ]:
!pip install fastai

In [ ]:
!pip install joblib

In [ ]:
!pip install python-gdcm

In [ ]:
!pip install scikit-image

In [ ]:
!pip install efficientnet

In [ ]:
# troubleshooting: compatible with keara 2.2.4
!pip install 'h5py==2.10.0'

In [ ]:
#%%
import os
from os import listdir, walk
#from os.path import join

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
import random

In [ ]:
start_time = time.time()

## Analysis directory
directory = "/home/u4574403/Project_Jane/StemCellReg2021/Oral cancer three-class classification with SoftLabels (True Final 4 INTER)"
# "D:\Project_Jane\StemCellReg2021\Oral cancer three-class classification with SoftLabels (Prev INTER without 140 and 410)"

os.chdir(directory)
os.getcwd()




In [ ]:
from collections import Counter, OrderedDict
#import customized functions
from CNN_Procedure import S1_Prep_RadomSplit as S1
from CNN_Procedure import S2_TrainSet_Prep as S2

In [ ]:
from CNN_Procedure import S3_Modeling as S3

In [ ]:
from CNN_Procedure import S4_Evaluation as S4

In [ ]:
#%%                                        I. Load Raw Data & Train_Test Split
#%%
data_ver =  "Hard with Soft_true_DS-OS" #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#"Hard only" #M1; Prep == None
#"Hard with Soft" #M2; Prep == "INTER"
#"Hard with Soft_OS" #M3; Prep== "OS_INTER"
#Hard with Soft_B-SMOTE #M4; Prep == "Bord_SMOTE_INTER"
#"Hard with Soft_DS-OS" #M5;  Prep == "DS_INTER"
Prep = "DS_INTER"                #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#%%                                        I.0 Input
## Data directory: Hard (Val / Test)
Val_folder = 'oral cancer 0521-0618_tag300_Val'
Val_path = "/home/u4574403/Project_Jane/StemCellReg2021/Data/" + Val_folder
#Define the splitted file names and their ratios (sum=1) for Val set
Val_split_ratio = OrderedDict({'train': 1.0}) 
#if test set inculded in the same folder: split_ratio =  OrderedDict({'train': 0.80, 'test': 0.20})

Test_folder = 'oral cancer 0521-0618_tag300_Test'
Test_path = "/home/u4574403/Project_Jane/StemCellReg2021/Data/" + Test_folder
Test_split_ratio = OrderedDict({'test': 1.0}) 

Labels = ["control", "5nMTG", 'NonCancerOral']
Labels_dic ={}
for i, label in enumerate(Labels):
    Labels_dic[label] = i



## Data directory: Intermediate Data (Val / Test) 
Val_INTER_folder = 'oral cancer inter stage-all_tag300_Val'
Val_INTER_path = "/home/u4574403/Project_Jane/StemCellReg2021/Data/" + Val_INTER_folder
Val_split_INTER_ratio = OrderedDict({'train': 0.8, 'val': 0.2}) 
#if test set inculded in the same folder: split_ratio =  OrderedDict({'train': 0.64, 'val': 0.16, 'test': 0.20})

Test_INTER_folder = 'oral cancer inter stage-all_tag300_Test'
Test_INTER_path = "/home/u4574403/Project_Jane/StemCellReg2021/Data/" + Test_INTER_folder
Test_split_INTER_ratio = OrderedDict({'test': 1.0}) 

Labels_INTER = ['20X_TG2-WT1', '20X_TG1-WT2', '20X_TG3-WT1', '20X_TG1-WT3'] #['TG2-WT1_0709', '20X_TG2-WT1', '20X_TG1-WT2', '20X_TG3-WT1', '20X_TG1-WT3'] 

In [ ]:
#%%                                        I.0 Input parameters for model training
# For custom model parameters: loss, Activaion function
Loss_name = 'mse' #'FocalLoss' 
Activation = 'softmax' #'softmax', 'sigmoid'
"""
for Loss_name = 'categorical_crossentropy'
S3.create_EfficientNetB3(Loss=Loss_name)
"""


#For integer m >=1, suppose the rartio - alpha0 : alpha1 : alph2 = m*x : x : x = (1-2/(m+2)) : 1/(m+2) : 1/(m+2)
#From binary classification result, alpha0 : alpha1 = m*x : x where m~ 5~6

gammas = [None]#[5, 5, 5]
alpha_lists = [[None,None,None]]



# For training set preprocessing
'''
if to resize image immediately in input stage (after Split_Data_Strafied): set " resize = say, (W, H) = (300, 300)"; o.w., None
  -> preprocess pairs (resize, Prep):((300, 300), None) or (None, None), (None, "CutMix"), (None, "Puzzle")
'''
# Prep = None                  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

## optional
if Prep== None:    ## M1:training with only hard labels 
    resize = (300, 300)
    other_info = None
if Prep== "INTER" or "OS_INTER": ##M2:"INTER": training with INTER (soft label)data;  M4:"OS_INTER": training with Over-Sampled INTER
    resize = (300, 300)
    other_info = None
if Prep=="CutMix_INTER": #"CutMix":
    resize = (300, 300)
    CutMix_state = 2 # 0: conventilanl; 1: only same label CutMixed; 2: only different label CutMixed
    other_info = CutMix_state  
if Prep == "SMOTE_INTER":
    resize = (300, 300)
    SMOTE_state = 1 # 0: conventilanl; 1: only different label synthesized; 
    other_info = SMOTE_state
if Prep == "Bord_SMOTE_INTER": #M3
    resize = (300, 300)
    other_info = None
if Prep=="Puzzle":
    resize = (300, 300) #for lung data: None;  for oral data: (1600, 1200)   # note: (W, H) for cv2 
    len_crop = 100 # for Puzzle
    other_info = len_crop
if Prep== "DS_INTER": ##M5"INTER": training with INTER (soft label)data;  "OS_INTER": training with Over-Sampled INTER
    resize = (300, 300)
    other_info = "OS" #None#



# for repetiontion
Trials = 1
K = 5

# for modeling
model_resize = None #input image size for model: EfficientNet B3
batch_size = 16 #16
epochs = 10


# Create output folder    
output_path = os.path.join(directory, f"output_{data_ver}_{Trials}-trials_{K}-fold_{Loss_name}")
if not os.path.exists(output_path):
    os.mkdir(output_path)


#%%

In [ ]:
#                                              II. K-fold Split  ~ IV. Model Training

#%%

# for K-fold cross validation
from sklearn.model_selection import StratifiedKFold

# for input data sequence generation before modeling
from keras.preprocessing.image import ImageDataGenerator

# for model fitting: generate data sequences    
datagen = ImageDataGenerator(
                                width_shift_range = 0.0,     # 水平平移
                                height_shift_range = 0.0,    # 垂直平移
                                rotation_range = 90,         # 0-180 任一角度旋轉
                                horizontal_flip = True,      # 任意水平翻轉
                                vertical_flip = True,        # 任意垂直翻轉
                                fill_mode = "constant",      # 在旋轉或平移時，有空隙發生，則空隙補常數
                                cval = 0                     # 設定常數值為 0
                                )
    
#%%                                            II.2 Repetition (i.e., iteration of multiple trials)  
#np.random.seed(214)       
for trial in range(Trials):
    trial = trial + 0 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    print(trial)
    #Random Split
    Val_split_data, Val_split_index = S1.Split_Data_Strafied(Val_path, Labels, Val_split_ratio, resize = resize) 
    Test_split_data, Test_split_index = S1.Split_Data_Strafied(Test_path, Labels, Test_split_ratio, resize = resize) 

    Val_split_data_INTER, Val_split_index_INTER = S1.Split_Data_Strafied(Val_INTER_path, Labels_INTER, Val_split_INTER_ratio, resize = resize) 
    Test_split_data_INTER, Test_split_index_INTER = S1.Split_Data_Strafied(Test_INTER_path, Labels_INTER, Test_split_INTER_ratio, resize = resize) 
       
    #Extract and name corresponding data - hard label ----------------------------------------------------------
    train_data = np.concatenate([split['train'] for label, split in Val_split_data.items()], 0)
    train_label = np.concatenate([np.zeros(len(split['train']), dtype=np.int_) + Labels_dic[label] for label, split in Val_split_data.items()], 0).astype(np.int_)
    train_index = [x  for label, split in Val_split_index.items() for x in split['train']]

    test_data = np.concatenate([split['test'] for label, split in Test_split_data.items()], 0)
    test_label = np.concatenate([np.zeros(len(split['test']), dtype=np.int_) + Labels_dic[label] for label, split in Test_split_data.items()], 0).astype(np.int_)
    test_index = [x  for label, split in Test_split_index.items() for x in split['test']]
    np.save(os.path.join(output_path, 'test_label.npy'), test_label)

    test_data = S1.Resize_Prep(test_data, model_resize) # resize testing set at the beginning, for model input


    #Extract and name corresponding data - soft label ------------------------------------------------------------    
    train_data_INTER = np.concatenate([split['train'] for label, split in Val_split_data_INTER.items() ], 0)       
    train_label_INTER = np.concatenate( [np.array([[1/3, 2/3, 0]]*len(Val_split_index_INTER[Labels_INTER[0]]['train'])),
                                         np.array([[2/3, 1/3, 0]]*len(Val_split_index_INTER[Labels_INTER[1]]['train'])),
                                         np.array([[1/4, 3/4, 0]]*len(Val_split_index_INTER[Labels_INTER[2]]['train'])),
                                         np.array([[3/4, 1/4, 0]]*len(Val_split_index_INTER[Labels_INTER[3]]['train'])) ], axis=0
                                        )
    train_index_INTER = [x  for label, split in Val_split_index_INTER.items() for x in split['train']] #before x in split[] > if label not in [Labels_INTER[i]]

    val_data_INTER = np.concatenate([split['val'] for label, split in Val_split_data_INTER.items() ], 0)
    val_label_INTER = np.concatenate( [np.array([[1/3, 2/3, 0]]*len(Val_split_index_INTER[Labels_INTER[0]]['val'])),
                                       np.array([[2/3, 1/3, 0]]*len(Val_split_index_INTER[Labels_INTER[1]]['val'])),
                                       np.array([[1/4, 3/4, 0]]*len(Val_split_index_INTER[Labels_INTER[2]]['val'])),
                                       np.array([[3/4, 1/4, 0]]*len(Val_split_index_INTER[Labels_INTER[3]]['val'])) ], axis=0
                                        )
    val_index_INTER = [x  for label, split in Val_split_index_INTER.items() for x in split['val']]

    test_data_INTER = np.concatenate([split['test'] for label, split in Test_split_data_INTER.items() ], 0)
    test_label_INTER = np.concatenate( [np.array([[1/3, 2/3, 0]]*len(Test_split_index_INTER[Labels_INTER[0]]['test'])),
                                        np.array([[2/3, 1/3, 0]]*len(Test_split_index_INTER[Labels_INTER[1]]['test'])),
                                        np.array([[1/4, 3/4, 0]]*len(Test_split_index_INTER[Labels_INTER[2]]['test'])),
                                        np.array([[3/4, 1/4, 0]]*len(Test_split_index_INTER[Labels_INTER[3]]['test'])) ], axis=0
                                        )
    test_index_INTER = [x  for label, split in Test_split_index_INTER.items() for x in split['test']]
    np.save(os.path.join(output_path, 'test_label_INTER.npy'), test_label_INTER)
    
    test_data_INTER = S1.Resize_Prep(test_data_INTER, model_resize) # resize testing set at the beginning, for model input
    
    #save sampel size
    val_INTER_n = {}
    for i in range(len(Val_split_index_INTER)):
        val_INTER_n[i] = len(Val_split_index_INTER[Labels_INTER[i]]['val'])
    test_INTER_n = {}
    for i in range(len(Test_split_index_INTER)):
        test_INTER_n[i] = len(Test_split_index_INTER[Labels_INTER[i]]['test'])

        
    
    #save test indices for each trial ------------------------------------------------------------------------
    with open(os.path.join(output_path, f'test_index_{Prep}-{other_info}_t{trial}.csv'), 'w') as temp_file:
        for text in test_index:
            temp_file.write(str(text)+'\n')
    with open(os.path.join(output_path, f'test_index_INTER_{Prep}-{other_info}_t{trial}.csv'), 'w') as temp_file:
        for text in test_index_INTER:
            temp_file.write(str(text)+'\n')
                
    del Val_split_data, Val_split_index, Test_split_data, Test_split_index, Val_split_data_INTER, Val_split_index_INTER, Test_split_data_INTER, Test_split_index_INTER
        
        
    skf = StratifiedKFold(n_splits=K, shuffle=True)

        #%%                                            II.2 K-fold Split
            
    for k, (train_skf_indices, val_indices) in enumerate(skf.split(train_data, train_label)):
        print(f"\n Current trial:\t " + str(trial+1) + f"/{Trials}...")
        print(f"\n Training on fold " + str(k+1) + f"/{K}...")
            #print("TRAIN:", train_skf_indices, "\n TEST:", val_indices)
            
        
        # Preprocess (if any) to k-fold training dataset
        if Prep == None: # train_skf_data + INTER data
            print(f"Training set preprocessing: {Prep}")
                
            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
            
            N_exp = 0
            N_exp2 = 0
            N_exp3 = 0
            N_exp4 = 0                 
        
        if Prep == "INTER": # train_skf_data + INTER data
            print(f"Training set preprocessing: {Prep}")
                
            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
                
            train_skf_data_INTER = S1.Resize_Prep(train_data_INTER, model_resize) 
                
            # Concatenate original and INTER data
            train_skf_data =  np.concatenate([train_skf_data, train_skf_data_INTER], 0)# ready for model input
            train_skf_label = np.concatenate([train_skf_label, train_label_INTER], 0) # one-hot encoder 
            
            N_exp = 0
            N_exp2 = 0
            N_exp3 = 0
            N_exp4 = 0

            del train_skf_data_INTER


        if Prep == "OS_INTER": # train_skf_data + INTER data
            print(f"Training set preprocessing: {Prep}")
            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
            
            train_skf_data_INTER = S1.Resize_Prep(train_data_INTER, model_resize) 
            
            #prepare data for OS ---------------------------------------------------------------------------------------------                          
            #reample label_lists[i] with size = N_exp_list[i]
            N_exp  = 128*2
            N_exp2 = 128*2
            N_exp3 = 128
            N_exp4 = 128*2

            N_exp_list = [N_exp, N_exp2, N_exp3, N_exp4]
            label_lists = [[3/4, 1/4, 0], [1/4, 3/4, 0], [2/3, 1/3, 0], [1/3, 2/3, 0]]
            
            data_OS = np.zeros((0,resize[0],resize[1],3))
            label_OS_m = np.zeros((0, len(Labels)))
            for N_exp0, label_list in zip(N_exp_list, label_lists):
                #print(n,'\t', label_list)
                data_OS0, label_OS_m0 = S2.Select_Resample(train_skf_data_INTER, train_label_INTER, label_list, N_exp0)                
                data_OS = np.concatenate([data_OS, data_OS0], 0)
                label_OS_m = np.concatenate([label_OS_m, label_OS_m0])
                
            # Concatenate original,INTER and SMOTE data --------------------------------------------------------------
            train_skf_data =  np.concatenate([train_skf_data, train_skf_data_INTER, data_OS], 0)# ready for model input
            train_skf_label = np.concatenate([train_skf_label, train_label_INTER, label_OS_m], 0) # one-hot encoder 
            del train_skf_data_INTER, data_OS, label_OS_m

        if Prep == "DS_INTER": # train_skf_data + INTER data 
            print(f"Training set preprocessing: {Prep}")
            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
            
            train_skf_data_INTER = S1.Resize_Prep(train_data_INTER, model_resize) 
            
            img_shape = train_skf_data.shape[1:]
            
            ## Operation (1): Down-Sample [2/3, 1/3, 0] V  vs [1/3, 2/3, 0] V
            data_list = [train_skf_data_INTER, train_skf_data_INTER]
            target_list = [train_label_INTER, train_label_INTER]
            label_lists = [[2/3, 1/3, 0], [1/3, 2/3, 0]] # output labels later will be [0, 1] correspondingly                 
            #DS preprocessing            
            data_concat1, target_concat_m1, N_concat = S2.DS_Prep(data_list, target_list, label_lists, img_shape=img_shape, sampling_strategy="auto")
            N210 = N_concat[0]; N120 = N_concat[1]

                        
            ## Operation (2): Down-Sample [2/3, 1/3, 0] V  vs [3/4, 1/4, 0] V
            data_list = [train_skf_data_INTER, train_skf_data_INTER]
            target_list = [train_label_INTER, train_label_INTER]
            label_lists = [[2/3, 1/3, 0], [3/4, 1/4, 0]] # output labels later will be [0, 1] correspondingly                 
            #DS preprocessing            
            data_concat2, target_concat_m2, N_concat = S2.DS_Prep(data_list, target_list, label_lists, img_shape=img_shape, sampling_strategy="all")

            data_concat2 = S2.Select_subset(data_concat2, target_concat_m2, label_lists[1])
            target_concat_m2 = np.array(label_lists[1]*N_concat[1]).reshape((-1, len(Labels)))
            N310 = N_concat[1]

            
            ## Operation (3): Down-Sample [4/5, 1/5, 0]  vs [3/4, 1/4, 0] V
            data_list = [train_skf_data_INTER, train_skf_data_INTER]
            target_list = [train_label_INTER, train_label_INTER]
            label_lists = [[1/3, 2/3, 0], [1/4, 3/4, 0]] # output labels later will be [0, 1] correspondingly                 
            #DS preprocessing            
            data_concat3, target_concat_m3, N_concat = S2.DS_Prep(data_list, target_list, label_lists, img_shape=img_shape, sampling_strategy="all")

            data_concat3 = S2.Select_subset(data_concat3, target_concat_m3, label_lists[1])
            target_concat_m3 = np.array(label_lists[1]*N_concat[1]).reshape((-1, len(Labels)))
            N130 = N_concat[1]
             
            
            ## Operation (0): Selet Data without Down-Sampling [1,0,0] vs [0,1,0] vs [0,0,1]
            data_list = [train_skf_data, train_skf_data, train_skf_data]
            target_list = [train_skf_label, train_skf_label, train_skf_label]
            label_lists = [[1,0,0], [0,1,0], [0,0,1]] # output labels later will be [0, 1, 2] correspondingly                 
                
            data_concat_flat, target_concat, N_concat = S2.Select_subset_flatten(data_list, target_list, label_lists )
            data_concat0, target_concat_m0 = S2.ReshapeX_TransY(data_concat_flat, target_concat, label_lists, img_shape=img_shape)

            DS_N = pd.DataFrame([OrderedDict({'N310': N310, 'N130': N130, 'N210': N210, 'N120': N120})])           
            DS_N["K-fold"] = k
            DS_N["Trial"] = trial
            
            
            if other_info == "OS":
                print(f"Plus training set preprocessing: {Prep}")
                
                train_skf_data = data_concat0 # ready for model input
                train_skf_label = target_concat_m0 # one-hot encoder
            
                train_skf_data_INTER_s = np.concatenate([data_concat1, data_concat2, data_concat3], 0)# ready for model input
                train_label_INTER_s = np.concatenate([target_concat_m1, target_concat_m2, target_concat_m3], 0) # one-hot encoder 
            
                #prepare data for OS ---------------------------------------------------------------------------------------------                          
                #reample label_lists[i] with size = N_exp_list[i]
                N_exp  = 128*2
                N_exp2 = 128*2
                N_exp3 = 128
                N_exp4 = 128*2
                N_exp_list = [N_exp, N_exp2, N_exp3, N_exp4]
                label_lists = [[3/4, 1/4, 0], [1/4, 3/4, 0], [2/3, 1/3, 0], [1/3, 2/3, 0]]
                
                data_OS = np.zeros((0,resize[0],resize[1],3))
                label_OS_m = np.zeros((0, len(Labels)))
                for N_exp0, label_list in zip(N_exp_list, label_lists):
                    #print(n,'\t', label_list)
                    data_OS0, label_OS_m0 = S2.Select_Resample(train_skf_data_INTER_s, train_label_INTER_s, label_list, N_exp0)                
                    data_OS = np.concatenate([data_OS, data_OS0], 0)
                    label_OS_m = np.concatenate([label_OS_m, label_OS_m0])
                
                # Concatenate original,INTER and SMOTE data --------------------------------------------------------------
                train_skf_data =  np.concatenate([train_skf_data, train_skf_data_INTER_s, data_OS], 0)# ready for model input
                train_skf_label = np.concatenate([train_skf_label, train_label_INTER_s, label_OS_m], 0) # one-hot encoder 
                del train_skf_data_INTER_s,train_label_INTER_s, data_OS, label_OS_m

            elif other_info == None:
                N_exp, N_exp2, N_exp3, N_exp4  = None, None, None, None
                # Concatenate original,INTER and SMOTE data --------------------------------------------------------------
                train_skf_data =  np.concatenate([data_concat0, data_concat1, data_concat2, data_concat3], 0)# ready for model input
                train_skf_label = np.concatenate([target_concat_m0, target_concat_m1, target_concat_m2, target_concat_m3], 0) # one-hot encoder 


            del data_concat0, data_concat1, data_concat2, data_concat3
            del target_concat_m0, target_concat_m1, target_concat_m2, target_concat_m3
            del data_list, target_list, label_lists, N_concat

        
        if Prep == "Bord_SMOTE_INTER": # train_skf_data + INTER data 
            print(f"Training set preprocessing: {Prep}")
            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
            
            train_skf_data_INTER = S1.Resize_Prep(train_data_INTER, model_resize) 

            #prepare data for SMOTE (with Mixeup)---------------------------------------------------------------------------------------------                          
            ## Case (1): [1,0,0] vs [3/4, 1/4, 0]
            N_exp  = 256
            label_lists = [[1,0,0], [3/4, 1/4, 0]] 
              
            data_SMOTE01 = S2.Select_subset(train_skf_data_INTER, train_label_INTER, label_list=label_lists[1]) 
            data_SMOTE00, _ = S2.Select_Resample(train_skf_data, train_skf_label, label_list=label_lists[0], N_exp=N_exp+len(data_SMOTE01)) #S2.Select_Resample(train_skf_data, train_skf_label, label_list=label_lists[0], N_exp=N_exp)        
            data_SMOTE0 = np.concatenate([data_SMOTE00, data_SMOTE01 ]) #data_SMOTE02
            #transform the dataset
            data_SMOTE0_flat = data_SMOTE0.reshape((len(data_SMOTE0), -1))
            label_SMOTE0_t = np.concatenate([ np.array([0]*len(data_SMOTE00)), np.array([1]*len(data_SMOTE01)) ])
            print(Counter(label_SMOTE0_t))
            oversampler = S2.BorderlineSMOTE(sampling_strategy="auto", kind='borderline-1')                #{1: N_exp + len(data_SMOTE01)}
            data_SMOTE0_flat2, label_SMOTE0_t2 = oversampler.fit_resample(data_SMOTE0_flat, label_SMOTE0_t)
            print(Counter(label_SMOTE0_t2))
            
            data_SMOTE0_flat2 = data_SMOTE0_flat2[label_SMOTE0_t2==1]
            data_SMOTE = data_SMOTE0_flat2.reshape(data_SMOTE00.shape)
            label_SMOTE_m = np.array([label_lists[1]*(N_exp+len(data_SMOTE01))]).reshape((-1, 3))                            
            del data_SMOTE00, data_SMOTE01, data_SMOTE0, data_SMOTE0_flat, data_SMOTE0_flat2, label_SMOTE0_t, label_SMOTE0_t2

            ## Case (2): [0,1,0] vs [1/4, 3/4, 0]
            N_exp2  = 256
            label_lists = [[0,1,0] , [1/4, 3/4, 0]] 
              
            data_SMOTE01 = S2.Select_subset(train_skf_data_INTER, train_label_INTER, label_list=label_lists[1])            
            data_SMOTE00, _ = S2.Select_Resample(train_skf_data, train_skf_label, label_list=label_lists[0], N_exp=N_exp2+len(data_SMOTE01)) #S2.Select_Resample(train_skf_data, train_skf_label, label_list=label_lists[0], N_exp=N_exp2) #
            data_SMOTE0 = np.concatenate([data_SMOTE00, data_SMOTE01 ]) #data_SMOTE02
            #transform the dataset
            data_SMOTE0_flat = data_SMOTE0.reshape((len(data_SMOTE0), -1))
            label_SMOTE0_t = np.concatenate([ np.array([0]*len(data_SMOTE00)), np.array([1]*len(data_SMOTE01)) ])
            print(Counter(label_SMOTE0_t))
            oversampler = S2.BorderlineSMOTE(sampling_strategy="auto", kind='borderline-1')                #{1: N_exp + len(data_SMOTE01)}
            data_SMOTE0_flat2, label_SMOTE0_t2 = oversampler.fit_resample(data_SMOTE0_flat, label_SMOTE0_t)
            print(Counter(label_SMOTE0_t2))
                            
            data_SMOTE0_flat2 = data_SMOTE0_flat2[label_SMOTE0_t2==1]
            data_SMOTE2 = data_SMOTE0_flat2.reshape(data_SMOTE00.shape)
            label_SMOTE_m2 = np.array([label_lists[1]*(N_exp2+len(data_SMOTE01))]).reshape((-1, 3))                            
            del data_SMOTE00, data_SMOTE01, data_SMOTE0, data_SMOTE0_flat, data_SMOTE0_flat2, label_SMOTE0_t, label_SMOTE0_t2

            ## Case (3): [3/4, 1/4, 0] vs [2/3, 1/3, 0]
            N_exp3 = 128
            label_lists = [[1,0,0], [2/3, 1/3, 0]] 
              
            data_SMOTE01 = S2.Select_subset(train_skf_data_INTER, train_label_INTER, label_list=label_lists[1])
            data_SMOTE00, _ = S2.Select_Resample(train_skf_data, train_skf_label, label_list=label_lists[0], N_exp=N_exp3+len(data_SMOTE01))
            data_SMOTE0 = np.concatenate([data_SMOTE00, data_SMOTE01 ]) #data_SMOTE02
            #transform the dataset
            data_SMOTE0_flat = data_SMOTE0.reshape((len(data_SMOTE0), -1))
            label_SMOTE0_t = np.concatenate([ np.array([0]*len(data_SMOTE00)), np.array([1]*len(data_SMOTE01)) ])
            print(Counter(label_SMOTE0_t))
            oversampler = S2.BorderlineSMOTE(sampling_strategy='auto', kind='borderline-1')                #{1: N_exp + len(data_SMOTE01)}
            data_SMOTE0_flat2, label_SMOTE0_t2 = oversampler.fit_resample(data_SMOTE0_flat, label_SMOTE0_t)
            print(Counter(label_SMOTE0_t2))
                            
            data_SMOTE0_flat2 = data_SMOTE0_flat2[label_SMOTE0_t2==1]
            data_SMOTE3 = data_SMOTE0_flat2.reshape(data_SMOTE00.shape)
            label_SMOTE_m3 = np.array([label_lists[1]*(N_exp3+len(data_SMOTE01))]).reshape((-1, 3))                            
            del data_SMOTE00, data_SMOTE01, data_SMOTE0, data_SMOTE0_flat, data_SMOTE0_flat2, label_SMOTE0_t, label_SMOTE0_t2


            ## Case (4): [1/4, 3/4, 0] vs [1/3, 2/3, 0]
            N_exp4 = 256
            label_lists = [[1/4, 3/4, 0], [1/3, 2/3, 0]] 
            
            data_SMOTE01 = S2.Select_subset(train_skf_data_INTER, train_label_INTER, label_list=label_lists[1])
            data_SMOTE00, _ = S2.Select_Resample(train_skf_data_INTER, train_label_INTER, label_list=label_lists[0], N_exp=N_exp4+len(data_SMOTE01))
            data_SMOTE0 = np.concatenate([data_SMOTE00, data_SMOTE01 ]) #data_SMOTE02
            #transform the dataset
            data_SMOTE0_flat = data_SMOTE0.reshape((len(data_SMOTE0), -1))
            label_SMOTE0_t = np.concatenate([ np.array([0]*len(data_SMOTE00)), np.array([1]*len(data_SMOTE01)) ])
            print(Counter(label_SMOTE0_t))
            oversampler = S2.BorderlineSMOTE(sampling_strategy='auto', kind='borderline-1')                #{1: N_exp + len(data_SMOTE01)}
            data_SMOTE0_flat2, label_SMOTE0_t2 = oversampler.fit_resample(data_SMOTE0_flat, label_SMOTE0_t)
            print(Counter(label_SMOTE0_t2))

            data_SMOTE0_flat2 = data_SMOTE0_flat2[label_SMOTE0_t2==1]
            data_SMOTE4 = data_SMOTE0_flat2.reshape(data_SMOTE00.shape)
            label_SMOTE_m4 = np.array([label_lists[1]*(N_exp4+len(data_SMOTE01))]).reshape((-1, 3))                            
            del data_SMOTE00, data_SMOTE01, data_SMOTE0, data_SMOTE0_flat, data_SMOTE0_flat2, label_SMOTE0_t, label_SMOTE0_t2
                            

            # Concatenate original,INTER and SMOTE data --------------------------------------------------------------
            train_skf_data =  np.concatenate([train_skf_data, data_SMOTE, data_SMOTE2, data_SMOTE3, data_SMOTE4], 0)# ready for model input
            train_skf_label = np.concatenate([train_skf_label, label_SMOTE_m, label_SMOTE_m2, label_SMOTE_m3, label_SMOTE_m4], 0) # one-hot encoder 
            del train_skf_data_INTER, data_SMOTE, data_SMOTE2, data_SMOTE3, data_SMOTE4
            del label_SMOTE_m, label_SMOTE_m2, label_SMOTE_m3, label_SMOTE_m4


        """  
        if Prep == "SMOTE_INTER":
            print(f"Training set preprocessing: {Prep}{other_info}")

            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
            
            train_skf_data_INTER = S1.Resize_Prep(train_data_INTER, model_resize) 
            
            #prepare data for SMOTE (with Mixeup)---------------------------------------------------------------------------------------------                          
            ## Case (1): [1,0,0] vs [4/5,1/5,0]
            N_exp = 849
            if N_exp >0:
                data_SMOTE, label_SMOTE = select_data2(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=0, label_val=[1, 4/5], Resample= False)
                data_SMOTE, label_SMOTE = S2.SMOTE_SoftLabel(data_SMOTE, label_SMOTE, state=SMOTE_state, n_sample=N_exp)
                label_SMOTE_m = np.zeros((len(label_SMOTE),len(Labels)))
                label_SMOTE_m[:,0] = label_SMOTE #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                label_SMOTE_m[:,1] = 1- label_SMOTE
                label_SMOTE_m[:,2] = 0
                del label_SMOTE
            else:
                data_SMOTE = np.zeros((0,300,300,3))
                label_SMOTE_m = np.zeros((0, 3))
            #output: [data_SMOTE, label_SMOTE_m]
            '''
            plt.hist(label_SMOTE)
            plt.savefig(f"Histogram of SMOTEd labels_state{SMOTE_state}_410.png", transparent=True)
            plt.close()
            '''            

            ## Case (2): [0,1,0] vs [1/5,4/5,0]
            N_exp2 = 0
            if N_exp2 >0:
                data_SMOTE2, label_SMOTE2 = select_data2(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=1, label_val=[1, 4/5], Resample= False)
                data_SMOTE2, label_SMOTE2 = S2.SMOTE_SoftLabel(data_SMOTE2, label_SMOTE2, state=SMOTE_state, n_sample=N_exp2)
                label_SMOTE_m2 = np.zeros((len(label_SMOTE2),len(Labels)))
                label_SMOTE_m2[:,0] = 1- label_SMOTE2 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                label_SMOTE_m2[:,1] = label_SMOTE2
                label_SMOTE_m2[:,2] = 0
                del label_SMOTE2
            else:
                data_SMOTE2 = np.zeros((0,300,300,3))
                label_SMOTE_m2 = np.zeros((0, 3))
            #output: [data_SMOTE2, label_SMOTE_m2]
            '''
            plt.hist(label_SMOTE2)
            plt.savefig(f"Histogram of SMOTEd labels_state{SMOTE_state}_140.png", transparent=True)
            plt.close()
            '''
            

            ## Case (3): [4/5,1/5,0] vs [3/4, 1/4, 0]
            N_exp3 = 0
            if N_exp3 >0:
                data_SMOTE3, label_SMOTE3 = select_data2(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=0, label_val=[4/5, 3/4], Resample= False)
                data_SMOTE3, label_SMOTE3 = S2.SMOTE_SoftLabel(data_SMOTE3, label_SMOTE3, state=SMOTE_state, n_sample=N_exp3)
                label_SMOTE_m3 = np.zeros((len(label_SMOTE3),len(Labels)))
                label_SMOTE_m3[:,0] = label_SMOTE3 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                label_SMOTE_m3[:,1] = 1- label_SMOTE3
                label_SMOTE_m3[:,2] = 0
                del label_SMOTE3
            else:
                data_SMOTE3 = np.zeros((0,300,300,3))
                label_SMOTE_m3 = np.zeros((0, 3))                
            #output: [data_SMOTE3, label_SMOTE_m3]
            '''
            plt.hist(label_SMOTE3)
            plt.savefig(f"Histogram of SMOTEd labels_state{SMOTE_state}_310.png", transparent=True)
            plt.close()
            '''
            

            ## Case (4): [1/5,4/5,0] vs [1/4, 3/4, 0]
            N_exp4 = 0
            if N_exp4 >0:
                data_SMOTE4, label_SMOTE4 = select_data2(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=1, label_val=[4/5, 3/4], Resample= False)
                data_SMOTE4, label_SMOTE4 = S2.SMOTE_SoftLabel(data_SMOTE4, label_SMOTE4, state=SMOTE_state, n_sample=N_exp4)
                label_SMOTE_m4 = np.zeros((len(label_SMOTE4),len(Labels)))
                label_SMOTE_m4[:,0] = 1- label_SMOTE4 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                label_SMOTE_m4[:,1] = label_SMOTE4
                label_SMOTE_m4[:,2] = 0
                del label_SMOTE4
            else:
                data_SMOTE4 = np.zeros((0,300,300,3))
                label_SMOTE_m4 = np.zeros((0, 3))
            #output: [data_SMOTE4, label_SMOTE_m4]
            '''
            plt.hist(label_SMOTE4)
            plt.savefig(f"Histogram of SMOTEd labels_state{SMOTE_state}_130.png", transparent=True)
            plt.close()
            '''
            
            
            ## Case (5): [3/4, 1/4, 0] vs [2/3, 1/3, 0]
            N_exp5 = 496
            if N_exp5 >0:
                data_SMOTE5, label_SMOTE5 = select_data2(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=0, label_val=[3/4 ,2/3], Resample= False)
                data_SMOTE5, label_SMOTE5 = S2.SMOTE_SoftLabel(data_SMOTE5, label_SMOTE5, state=SMOTE_state, n_sample=N_exp5)
                label_SMOTE_m5 = np.zeros((len(label_SMOTE5),len(Labels)))
                label_SMOTE_m5[:,0] = label_SMOTE5 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                label_SMOTE_m5[:,1] = 1- label_SMOTE5
                label_SMOTE_m5[:,2] = 0
                del label_SMOTE5
            else:
                data_SMOTE5 = np.zeros((0,300,300,3))
                label_SMOTE_m5 = np.zeros((0, 3))            
            #output: [data_SMOTE5, label_SMOTE_m5]
            '''
            plt.hist(label_SMOTE5)
            plt.savefig(f"Histogram of SMOTEd labels_state{SMOTE_state}_210.png", transparent=True)
            plt.close()
            '''
            

            ## Case (6): [1/4, 3/4, 0] vs [1/3, 2/3, 0]
            N_exp6 = 439
            if N_exp6 >0:
                data_SMOTE6, label_SMOTE6 = select_data2(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=1, label_val=[3/4 ,2/3], Resample= False)
                data_SMOTE6, label_SMOTE6 = S2.SMOTE_SoftLabel(data_SMOTE6, label_SMOTE6, state=SMOTE_state, n_sample=N_exp6)
                label_SMOTE_m6 = np.zeros((len(label_SMOTE6),len(Labels)))
                label_SMOTE_m6[:,0] = 1- label_SMOTE6 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                label_SMOTE_m6[:,1] = label_SMOTE6
                label_SMOTE_m6[:,2] = 0
                del label_SMOTE6
            else:
                data_SMOTE6 = np.zeros((0,300,300,3))
                label_SMOTE_m6 = np.zeros((0, 3))
            #output: [data_SMOTE6, label_SMOTE_m6]
            '''
            plt.hist(label_SMOTE6)
            plt.savefig(f"Histogram of SMOTEd labels_state{SMOTE_state}_120.png", transparent=True)
            plt.close()
            '''
            
            
            # Concatenate original,INTER and SMOTE data --------------------------------------------------------------
            train_skf_data =  np.concatenate([train_skf_data, train_skf_data_INTER, data_SMOTE, data_SMOTE2, data_SMOTE3, data_SMOTE4, data_SMOTE5, data_SMOTE6], 0)# ready for model input
            train_skf_label = np.concatenate([train_skf_label, train_label_INTER, label_SMOTE_m, label_SMOTE_m2, label_SMOTE_m3, label_SMOTE_m4, label_SMOTE_m5, label_SMOTE_m6], 0) # one-hot encoder 
            del train_skf_data_INTER, data_SMOTE, data_SMOTE2, data_SMOTE3, data_SMOTE4, data_SMOTE5, data_SMOTE6
            del label_SMOTE_m, label_SMOTE_m2, label_SMOTE_m3, label_SMOTE_m4, label_SMOTE_m5, label_SMOTE_m6


        if Prep == "CutMix_INTER":
            print(f"Training set preprocessing: {Prep}{other_info}")

            train_skf_data = S1.Resize_Prep(train_data[train_skf_indices], model_resize) # ready for model input
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder
            
            train_skf_data_INTER = S1.Resize_Prep(train_data_INTER, model_resize) 
            
            #prepare data for CutMix----------------------------------------------------------------------------------------------  
            ## Case (0): [1,0,0] vs [2/3, 1/3, 0]
            N_exp = 128
            data_CutMix, label_CutMix = select_data(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=0, N_exp= N_exp, Resample= True)
            data_CutMix, label_CutMix = S2.CutMix_Prep_gen(data_CutMix, label_CutMix, state=CutMix_state, model_resize=model_resize) # ready for model input            
            label_CutMix_m = np.zeros((len(label_CutMix),len(Labels)))
            label_CutMix_m[:,0] = label_CutMix #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
            label_CutMix_m[:,1] = 1- label_CutMix
            label_CutMix_m[:,2] = 0
            #output: [data_CutMix, label_CutMix_m]
            '''
            plt.hist(label_CutMix)
            plt.savefig(f"Histogram of CutMixed labels_state{CutMix_state}_210.png", transparent=True)
            plt.close()
            '''
            del label_CutMix

            ## Case (1): [0,1,0] vs [1/3, 2/3, 0]
            N_exp2 = 128
            data_CutMix2, label_CutMix2 = select_data(train_skf_data, train_skf_label, train_skf_data_INTER, train_label_INTER, label=1, N_exp= N_exp2, Resample= True)            
            data_CutMix2, label_CutMix2 = S2.CutMix_Prep_gen(data_CutMix2, label_CutMix2, state=CutMix_state, model_resize=model_resize) # ready for model input
            label_CutMix_m2 = np.zeros((len(label_CutMix2),len(Labels)))
            label_CutMix_m2[:,0] = 1- label_CutMix2 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
            label_CutMix_m2[:,1] = label_CutMix2
            label_CutMix_m2[:,2] = 0
            #output: [data_CutMix2, label_CutMix_m2]
            del label_CutMix2
           
            # Concatenate original,INTER and CutMix data --------------------------------------------------------------
            train_skf_data =  np.concatenate([train_skf_data, train_skf_data_INTER, data_CutMix, data_CutMix2], 0)# ready for model input
            train_skf_label = np.concatenate([train_skf_label, train_label_INTER, label_CutMix_m, label_CutMix_m2], 0) # one-hot encoder 
            del train_skf_data_INTER, data_CutMix, data_CutMix2, label_CutMix_m, label_CutMix_m2
    
        if Prep == "Puzzle":
            print(f"Training set preprocessing: {Prep}{other_info}")
            train_skf_data = train_data[train_skf_indices]
            train_skf_data = S2.Puzzle_Prep(train_skf_data, len_crop, model_resize)     # ready for model input          
            train_skf_label = np.eye(len(Labels))[train_label[train_skf_indices]] # one-hot encoder

        """ 
        #%%                                                 II.1 Parameters for Loss Function
        for alpha_list, gamma in zip(alpha_lists, gammas):
            print(f"\n\n alpha_list = {alpha_list}\t gamma={gamma}")
            
            #%%                                                IV. Model Training
           
            # call model
            model = S3.create_EfficientNetB3(n_class=len(Labels), Loss=Loss_name, Activation =Activation)
            print(f"\n Current parameters for loss function successful embedded - {Loss_name}: gamma={gamma}, alpha_list={alpha_list}\n")
            
            # Preprocess (if any) to k-fold validation dataset
            val_skf_data = S1.Resize_Prep(train_data[val_indices], model_resize) # ready for model input 
            val_skf_label = np.eye(len(Labels))[train_label[val_indices]] 
    
            # for model fitting: training with data sequences
            callbacks_list = S3.call_LearningRateScheduler(S3.step_decay, verbose=1)
            model_history = model.fit_generator(
                                                datagen.flow(train_skf_data, train_skf_label, batch_size = batch_size),
                                                steps_per_epoch = len(train_skf_indices) / batch_size,
                                                validation_data = (val_skf_data, val_skf_label),
                                                epochs = epochs,
                                                verbose = 1,
                                                callbacks = callbacks_list
                                                )
        
            # Save model
            model.save(os.path.join(output_path, f'EfficientNetB3_{Prep}{other_info}_t{trial}_K{k}.h5'))


            #%%                                      V. Evaluation phases   
            #%%
        
            #%%                                      V.1 Record History         
            # training process
            train_history = pd.DataFrame(model_history.history)
            train_history["epoch"] = list(range(epochs))
            train_history["K-fold"] = k 
            train_history["Trial"] = trial
            train_history["Focal_gamma"] = gamma
            train_history["Focal_alpha0"] = alpha_list[0]
            train_history["Focal_alpha1"] = alpha_list[1]
            train_history["N1"] = N_exp
            train_history["N2"] = N_exp2
            train_history["N3"] = N_exp3
            train_history["N4"] = N_exp4       
            #%%                                      V.2 Evaluation for metrics                  
            # confution matrix and Accuracy, sensitivity, specificity
            ## for  validation set of k-fold
            val_predict_prob = model.predict(val_skf_data)
            val_predict_label = np.argmax(val_predict_prob,axis=1) 
            if Activation == 'sigmoid':
                ### Row normalization (Prob sum up t0 1)
                val_predict_prob = val_predict_prob/val_predict_prob.sum(axis=1)[:, np.newaxis]
   
            report_val, overall_val = S4.Multiclass_metrics_OVR(y_pred_prob = val_predict_prob, y_pred_label = val_predict_label, 
                                                                y_true = train_label[val_indices], y_index_names = np.array(train_index)[val_indices], target_names = Labels)
            report_val['dataset'] = 'val'
            overall_val['dataset'] = 'val'
    
            ## for testing set
            test_predict_prob = model.predict(test_data)
            test_predict_label = np.argmax(test_predict_prob,axis=1)
            if Activation == 'sigmoid':
                ### Row normalization (Prob sum up t0 1)
                test_predict_prob = test_predict_prob/test_predict_prob.sum(axis=1)[:, np.newaxis]
            
            report_test, overall_test = S4.Multiclass_metrics_OVR(y_pred_prob = test_predict_prob, y_pred_label = test_predict_label, 
                                                                  y_true = test_label, y_index_names = test_index, target_names = Labels)
            report_test['dataset'] = 'test'
            overall_test['dataset'] = 'test'


            # concate results
            report = pd.concat([report_val, report_test], axis =0)
            report["Focal_gamma"] = gamma
            report["Focal_alpha0"] = alpha_list[0]
            report["Focal_alpha1"] = alpha_list[1]
            report["N1"] = N_exp
            report["N2"] = N_exp2
            report["N3"] = N_exp3
            report["N4"] = N_exp4
            report["Trial"] = trial
            report["K-fold"] = k
            
            overall = pd.DataFrame([overall_val, overall_test])
            overall["Focal_gamma"] = gamma
            overall["Focal_alpha0"] = alpha_list[0]
            overall["Focal_alpha1"] = alpha_list[1]
            overall["N1"] = N_exp
            overall["N2"] = N_exp2
            overall["N3"] = N_exp3
            overall["N4"] = N_exp4
            overall["Trial"] = trial
            overall["K-fold"] = k            
            
            ## save testing set prediction (of each fold of model) -------------------------------------------
            test_predict_prob_df = pd.DataFrame(test_predict_prob, columns=Labels)
            test_predict_prob_df["accuracy"] = overall_val['accuracy']
            test_predict_prob_df["f1-score_macro"] = overall_val['f1-score_macro']
            test_predict_prob_df["Trial"] = trial
            test_predict_prob_df["K-fold"] = k
            
            
            del report_val, report_test
            

            """        
            #%%                                  V.3 Evaluation for ROC-AUC    
            ##　for  validation set of k-fold
            fpr, tpr, thresholds, roc_auc = S4.ROC_AUC(train_label[val_indices], val_predict_prob)      
            val_roc = OrderedDict({"fpr": fpr[1], "tpr": tpr[1], "thresholds": thresholds[1]})
            val_roc = pd.DataFrame.from_dict(val_roc)
            val_roc["Dataset"] = "val"
            val_roc["order"] = val_roc.index
      
            ##　for test set
            fpr, tpr, thresholds, test_roc_auc = S4.ROC_AUC(test_label, test_predict_prob)
            test_roc = OrderedDict({"fpr": fpr[1], "tpr": tpr[1], "thresholds": thresholds[1]})
            test_roc = pd.DataFrame.from_dict(test_roc)
            test_roc["Dataset"] = "test"  
            test_roc["order"] = test_roc.index
    
            roc_df = pd.concat([val_roc, test_roc], axis = 0, ignore_index=True)
            roc_df["K-fold"] = k
            roc_df["Trial"] = trial
            roc_df["Focal_gamma"] = gamma
            roc_df["Focal_alpha0"] = alpha0
            """
            del val_predict_prob, val_predict_label, test_predict_prob, test_predict_label
            #%%                                      V.4 Evaluation for metrics: Intermediate data
            #Prediction of Intermediate data for validation set
            val_new_predict_prob = model.predict(val_data_INTER)
            
            if Activation == 'sigmoid':
                ### Row normalization (Prob sum up t0 1)
                val_new_predict_prob = val_new_predict_prob/val_new_predict_prob.sum(axis=1)[:, np.newaxis]
            
            RMSE = math.sqrt( ((val_label_INTER - val_new_predict_prob)**2).sum()/len(val_new_predict_prob) )
            
            #Prediction of Intermediate data for testing set
            test_new_predict_prob = model.predict(test_data_INTER) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            if Activation == 'sigmoid':
                ### Row normalization (Prob sum up t0 1)
                test_new_predict_prob = test_new_predict_prob/test_new_predict_prob.sum(axis=1)[:, np.newaxis]
                
            RMSE_test = math.sqrt( ((test_label_INTER - test_new_predict_prob)**2).sum()/len(test_new_predict_prob) )

            inter_dict = OrderedDict({"Focal_gamma": gamma, "Focal_alpha0": alpha_list[0], "Focal_alpha1": alpha_list[1],
                                      "N1": N_exp, "N2": N_exp2, "N3": N_exp3, "N4": N_exp4,
                                      "Trial": trial, "K-fold": k, "RMSE_val": RMSE, "RMSE_test": RMSE_test})
            for ii in range(len(Labels)):
                inter_dict[f'Mean_P{str(ii)}_val'] = val_new_predict_prob[:,ii].mean()
            for ii in range(len(Labels)):
                inter_dict[f'Mean_P{str(ii)}_test'] = test_new_predict_prob[:,ii].mean()  

            ## save INTER testing set prediction (of each fold of model)  -------------------------------------------
            test_new_predict_prob_df = pd.DataFrame(test_new_predict_prob, columns=Labels)
            test_new_predict_prob_df["accuracy"] = overall_val['accuracy']
            test_new_predict_prob_df["f1-score_macro"] = overall_val['f1-score_macro']
            test_new_predict_prob_df["RMSE"] = RMSE
            test_new_predict_prob_df["Trial"] = trial
            test_new_predict_prob_df["K-fold"] = k
            
            # Calculate mean prob. for differenct INTER data -------------------------------------------------------------
            ##for validation set
            val_new_predict_prob = pd.DataFrame(val_new_predict_prob)
            val_new_predict_prob.columns = ['Pr_'+i for i in Labels]
            val_new_predict_prob['C0>C1'] = (val_new_predict_prob.iloc[:,0] > val_new_predict_prob.iloc[:,1])*1
            val_new_predict_prob['C0<C1'] = 1-val_new_predict_prob['C0>C1']
            val_new_predict_prob['INTER_type'] = 0
            val_INTER_n[0] = 0
            val_INTER_n_cum = pd.DataFrame([val_INTER_n]).cumsum(axis=1).values
            for i in range(len(val_INTER_n)-1): 
                val_new_predict_prob.loc[ val_INTER_n_cum[0, i]:val_INTER_n_cum[0, i+1],'INTER_type'] = i+1
                
            val_M_INTER = val_new_predict_prob.groupby(['INTER_type']).mean()
            
            ## for testing set
            test_new_predict_prob = pd.DataFrame(test_new_predict_prob)
            test_new_predict_prob.columns = ['Pr_'+i for i in Labels]
            test_new_predict_prob['C0>C1'] = (test_new_predict_prob.iloc[:,0] > test_new_predict_prob.iloc[:,1])*1
            test_new_predict_prob['C0<C1'] = 1-test_new_predict_prob['C0>C1']
            test_new_predict_prob['INTER_type'] = 0
            test_INTER_n[0] = 0
            test_INTER_n_cum = pd.DataFrame([test_INTER_n]).cumsum(axis=1).values
            for i in range(len(test_INTER_n)-1): 
                test_new_predict_prob.loc[ test_INTER_n_cum[0, i]:test_INTER_n_cum[0, i+1],'INTER_type'] = i+1
                
            test_M_INTER = test_new_predict_prob.groupby(['INTER_type']).mean()            
            
            
            val_M_INTER['dataset'] = 'val'
            test_M_INTER['dataset'] = 'test'

            # concate results
            M_INTER = pd.concat([val_M_INTER, test_M_INTER], axis =0)
            M_INTER["Focal_gamma"] = gamma
            M_INTER["Focal_alpha0"] = alpha_list[0]
            M_INTER["Focal_alpha1"] = alpha_list[1]
            M_INTER["N1"] = N_exp
            M_INTER["N2"] = N_exp2
            M_INTER["N3"] = N_exp3
            M_INTER["N4"] = N_exp4

            M_INTER["Trial"] = trial
            M_INTER["K-fold"] = k
            
            
            del overall_val, overall_test 
            del val_M_INTER, test_M_INTER, val_new_predict_prob, test_new_predict_prob

            #%%                                        VII. Record and Output to files
            #%%
            # Export all results directory after all iterations
            with open(os.path.join(output_path, f'Evaluation_{Prep}{other_info}_inter_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as evalF_inter:
                pd.DataFrame([inter_dict]).to_csv(evalF_inter, mode='a',header = evalF_inter.tell()==0, index=False)
            
            with open(os.path.join(output_path, f'Evaluation_{Prep}{other_info}_overall_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as evalF_overall:
                overall.to_csv(evalF_overall, mode='a',header = evalF_overall.tell()==0, index=False)
                
            with open(os.path.join(output_path , f'Evaluation_{Prep}{other_info}_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as evalF:
                report.to_csv(evalF, mode='a', header = evalF.tell()==0, index=True)
        
            with open(os.path.join(output_path, f'History_{Prep}{other_info}_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as historyF:
                train_history.to_csv(historyF, mode='a', header = historyF.tell()==0, index=False)

            with open(os.path.join(output_path , f'Evaluation_{Prep}{other_info}_inter_M_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as evalF_inter_M:
                M_INTER.to_csv(evalF_inter_M, mode='a', header = evalF_inter_M.tell()==0, index=True)

            if Prep == "DS_INTER":
                with open(os.path.join(output_path, f'DS_N_{Prep}{other_info}_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as DS_F:
                    DS_N.to_csv(DS_F, mode='a', header = DS_F.tell()==0, index=False)
 
            with open(os.path.join(output_path , f'Evaluation_{Prep}{other_info}_TestPred_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as evalF_test:
                test_predict_prob_df.to_csv(evalF_test, mode='a', header = evalF_test.tell()==0, index=True)
            with open(os.path.join(output_path , f'Evaluation_{Prep}{other_info}_TestPred_inter_batch-{batch_size}_epoch-{epochs}.csv'), "a", newline='\n') as evalF_test:
                test_new_predict_prob_df.to_csv(evalF_test, mode='a', header = evalF_test.tell()==0, index=True)
       
        del train_skf_data, val_skf_data, model, callbacks_list, model_history, train_history, report, M_INTER, test_predict_prob_df, test_new_predict_prob_df

    del train_data, test_data, train_label, test_label, train_index, test_index




end_time = time.time()
print("Output completed, time spent (in hours):", (end_time - start_time)/3600)



"""
#%%                                        VIII. Model prediction on new testing set
#%%

#%%                                  I.1 Load trained model from .h5
from keras.models import load_model
# for model
#from CNN_Procedure import S3_Modeling as S3
#model = load_model(f'EfficientNetB3_lung_TVGH_{data_ver}.h5') # without customized loss function

# For customized loss funciton (before loading model)
trial = 0
k= 0

#import keras.losses
#keras.losses.focal_loss = S3.FocalLoss(gamma, alpha_list)

model = load_model(os.path.join(output_path, f'EfficientNetB3_NoneNone_t{str(trial)}_K{str(k)}.h5'))
#or other options
#model_3 = load_model(os.path.join(output_path, 'EfficientNetB3_NoneNone_b16_e5_t1_K3_a0.82_g5.h5'), custom_objects={'FocalLoss':S3.FocalLoss(gamma, alpha_list)})

'''
if you wish to just perform inference with your model and not further optimization or training your model,
 you can simply wish to ignore the loss function like this: model_2
model_2 = load_model(os.path.join(output_path, 'EfficientNetB3_NoneNone_b16_e5_t1_K3_a0.82_g5.h5'), compile=False)
'''

#%%                                   I.2 Load testing set
H, W =  300, 300 # 224,224# Input shape, defined by the model (model.input_shape)
resize = (H, W) 

data_folder = 'oral cancer inter stage-all_tag300'
origion_path = "D:\\Project_Jane\\StemCellReg2021\\Data\\" + data_folder


test_new_path = pd.read_csv(os.path.join(output_path, 'test_index_INTER_NoneNone_t0.csv'), header=None).values
test_new_path = [i[0] for i in test_new_path.tolist()]


Labels_INTER = ['TG2-WT1_0709', '20X_TG2-WT1_1021', '20X_TG1-WT2_1021'] #soft labels = [[1/3, 2/3, 0], [1/3, 2/3, 0], [2/3, 1/3, 0]]
test_new_data= []
for j, i in enumerate(test_new_path):
    print(j, i)
    if j < 40:
        Label_INTER = Labels_INTER[1]
    else:
        Label_INTER = Labels_INTER[2]
    print('current searched folder:', Label_INTER)    
    test_image = cv2.imread(os.path.join(origion_path, Label_INTER, i))[:,:,::-1]
    #test_image = cv2.resize(test_image, resize )
    test_new_data.append(test_image)
test_new_data= np.array(test_new_data)


#%%                                             I.1 Predition
#%% prediction
test_predict_prob = model.predict(test_new_data)
test_predict_label = np.argmax(test_predict_prob,axis=1)


test_predict_prob = pd.DataFrame(test_predict_prob)
test_predict_prob.columns = ['Pr_'+i for i in Labels]
test_predict_prob.index = test_new_path

test_predict_prob.to_csv(os.path.join(output_path, f'Pred_Prob_Oral_INTER_Data_{Prep}{other_info}_t{trial}_K{k}.csv'))

#%% summary statistics
test120 = test_predict_prob.iloc[:40]
test210 = test_predict_prob.iloc[40:]
print(test120.mean())
print(test210.mean())

print("test120: ratio of Pr_5nMTG>Pr_control:\t",sum(test120['Pr_5nMTG']> test120['Pr_control'])/40)
print("test210: ratio of Pr_control>Pr_5nMTG:\t",sum(test210['Pr_control']>test210['Pr_5nMTG'])/40)

#%% histogram
print("Mean probabilities:\n", test_predict_prob.mean(axis=0))
M = test_predict_prob['Pr_control'].mean()

   

eps = [0, -2, 0]    
colors = ['blue', 'orange', 'green']
plt.figure(figsize=(10,7))
for i, col in enumerate(test_predict_prob.columns):
    M = test_predict_prob[col].mean()
    test_predict_prob[col].hist(bins = 20, color = colors[i], range=[0,1], label =col, alpha =0.5) #bins=10, range=[0,0.5], color ='orange'
    plt.axvline(M, color = colors[i])
    plt.annotate(f'Mean={str(round(M,4))}',xy = (M, 0), xytext = (M, -4+eps[i]), arrowprops =dict(facecolor=colors[i])) #(M-0.06, -10)
plt.title(f"Histogram of Prob")  
plt.legend() 
plt.savefig(os.path.join(output_path, f'Histogram of Prob_t{trial}_K{k}.png'), transparent=True)
plt.close()   

    
colors = ['blue', 'orange', 'green']
plt.figure(figsize=(10,16))
plt.subplot(2,1,1)
eps = [0, 0, 0]
for i, col in enumerate(test_predict_prob.columns):
    M = test120[col].mean()
    test120[col].hist(bins = 20, color = colors[i], range=[0,1], label =col, alpha =0.5) #bins=10, range=[0,0.5], color ='orange'
    plt.axvline(M, color = colors[i])
    plt.annotate(f'Mean={str(round(M,4))}',xy = (M, 0), xytext = (M, -3+eps[i]), arrowprops =dict(facecolor=colors[i]), fontsize=6) #(M-0.06, -10)
plt.title(f"Histogram of Prob (Control: 5nMTG=1:2)")  
plt.legend() 

plt.subplot(2,1,2)
eps = [0, 0, 0]
for i, col in enumerate(test_predict_prob.columns):
    M = test210[col].mean()
    test210[col].hist(bins = 20, color = colors[i], range=[0,1], label =col, alpha =0.5) #bins=10, range=[0,0.5], color ='orange'
    plt.axvline(M, color = colors[i])
    plt.annotate(f'Mean={str(round(M,4))}',xy = (M, 0), xytext = (M, -3+eps[i]), arrowprops =dict(facecolor=colors[i]), fontsize=6) #(M-0.06, -10)
plt.title(f"Histogram of Prob (Control: 5nMTG=2:1)")  
plt.legend() 
plt.savefig(os.path.join(output_path, f'Histogram of Prob_t{trial}_K{k}_subplots.png'), transparent=True)
plt.close() 


#%% stacked-Bar Plot            # Check if modification needed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
test_predict_prob2 = test_predict_prob#.sort_values(['Pr_control'])
    
index = list(range(len(test_predict_prob2)))
fig, ax = plt.subplots()
ax.bar(index, test_predict_prob2['Pr_control'], label='Pr_control', align='edge', width=1.0)
ax.bar(index, test_predict_prob2['Pr_5nMTG'], bottom=test_predict_prob2['Pr_control'], label="Pr_5nMTG", align='edge', width=1.0)
ax.bar(index, test_predict_prob2['Pr_NonCancerHEK293T'], bottom = test_predict_prob2['Pr_control'] + test_predict_prob2['Pr_5nMTG'], label="Pr_NonCancerHEK293T", align='edge', width=1.0)
ax.legend(loc = 'lower right') #'best'
#plt.show()
plt.savefig(os.path.join(output_path, f'Stacked bar plot_t{trial}_K{k}.png'), transparent=True)
plt.close()


'''
#%%                                        IX. Preprocessing Visual Illustration
#%%
# illustrate image interpolation (for SMOTE / Mixup)
directory = "D:\Project_Jane\StemCellReg2021\Data"
X = cv2.imread(os.path.join(directory, "oral cancer inter stage-all_tag300\\20X_TG1-WT2_1021\\inter210_00.tif"))[:,:,::-1]
X_nn = cv2.imread(os.path.join(directory,"oral cancer 0521-0618_tag300\\control\\control_00.tif"))[:,:,::-1]

r = 0.24
X_Mixup = X + r*(X_nn-X) 
temp = np.round(X_Mixup,0)
temp=temp.astype(np.int_)

plt.imshow(temp)
plt.axis('off') 
plt.savefig("Mixup inter210_00 vs control_00.png", transparent = True)
'''
"""
